In [ ]:
!nvidia-smi

Tue Dec  1 10:49:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
!ls

%cd ./ResUNetplusplus
!ls 

Mounted at /content/gdrive
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
ResUNetplusplus
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus
'=1.0.7,'			   process_image_sample.py
'Copy of execution_script.ipynb'   __pycache__
 data				   README.md
 data_generator.py		   result
 deployment_model.py		   results.png
 execution_script.ipynb		   resume_training.py
 files				   Resunet++.png
 frozen_models			   resunet_pp_modified.py
 GFLOPs.ipynb			   resunet.py
 infer.py			   run_miouTesting.py
 metrics.py			   run_modified.py
 m_resunet_modified.py		   run.py
 m_resunet.py			   run_sample.py
 new_data			   segmentation_models.ipynb
 process_image_deployment.py	   unet.py
 process_image.py


In [ ]:
# !pip install git+https://github.com/qubvel/segmentation_models
# !pip install tensorflow==2.2.0
# !pip install -U segmentation-models==0.2.1 #install this
!pip install keras==2.3.1 #install this
!pip install tensorflow==2.2.0 #install this
### Then restart runtime
# !pip install keras_applications >=1.0.7,<=1.0.8
!pip install segmentation-models==1.0.1

     |████████████████████████████████| 378kB 5.7MB/s 
     |████████████████████████████████| 51kB 4.1MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 516.2MB 29kB/s 
     |████████████████████████████████| 3.0MB 39.9MB/s 
     |████████████████████████████████| 460kB 41.0MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
import segmentation_models as sm

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [ ]:
# from tensorflow import keras
model = sm.Unet('resnet34', classes=1, activation='softmax')
model.summary()

85524480/85521592 [==============================] - 191s 2us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 3 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, None, None, 3 9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, None, None, 6 9408        zero_padding2d_1[0][0]           
_________________________

In [ ]:
#!python3 process_image.py

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100
100% 100/100 [00:03<00:00, 26.61it/s]
100% 100/100 [00:03<00:00, 29.23it/s]
  4% 30/800 [02:09<3:22:54, 15.81s/it]

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from data_generator import DataGen
from unet import Unet
from resunet import ResUnet
# from m_resunet_modified import ResUnetPlusPlus
from resunet_pp_modified import ResUnetPlusPlus
from metrics import dice_coef, dice_loss, miou_coef, miou_loss


if __name__ == "__main__":
    ## Path
    file_path = "files/"
    model_path = "files/unet_miou.h5"
    model_name = "unet_miou"

    ## Create files folder
    try:
        os.mkdir("files")
    except:
        pass

    train_path = "new_data/Kvasir-SEG/train/"
    valid_path = "new_data/Kvasir-SEG/valid/"

    ## Training
    train_image_paths = glob(os.path.join(train_path, "images", "*"))
    train_mask_paths = glob(os.path.join(train_path, "masks", "*"))
    train_image_paths.sort()
    train_mask_paths.sort()

    # train_image_paths = train_image_paths[:2000]
    # train_mask_paths = train_mask_paths[:2000]

    ## Validation
    valid_image_paths = glob(os.path.join(valid_path, "images", "*"))
    valid_mask_paths = glob(os.path.join(valid_path, "masks", "*"))
    valid_image_paths.sort()
    valid_mask_paths.sort()

    ## Parameters
    image_size = 256
    batch_size = 8
    lr = 1e-4
    epochs = 10

    train_steps = len(train_image_paths)//batch_size
    valid_steps = len(valid_image_paths)//batch_size

    print("train steps:", train_steps , "valid_steps", valid_steps)

    ## Generator
    train_gen = DataGen(image_size, train_image_paths, train_mask_paths, batch_size=batch_size)
    valid_gen = DataGen(image_size, valid_image_paths, valid_mask_paths, batch_size=batch_size)

    ## Unet
    # arch = Unet(input_size=image_size)
    # model = arch.build_model()
    model = sm.Unet('seresnet18', classes=1, activation='sigmoid'
    # , encoder_weights='imagenet'
    )

    ## ResUnet
    # arch = ResUnet(input_size=image_size)
    # model = arch.build_model()

    ## ResUnet++
    # arch = ResUnetPlusPlus(input_size=image_size)
    # model = arch.build_model()

    optimizer = Nadam(lr)
    # metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2)]
    # model.compile(loss=dice_loss, optimizer=optimizer, metrics=metrics)
    metrics = [Recall(), Precision(), dice_coef, MeanIoU(num_classes=2),miou_coef]
    # model.compile(loss=miou_loss, optimizer=optimizer, metrics=metrics)
    model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
    )
    # model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=optimizer, metrics=metrics)

    csv_logger = CSVLogger(f"{file_path}{model_name}_{batch_size}_{epochs}.csv", append=False)
    checkpoint = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
    callbacks = [csv_logger, checkpoint, reduce_lr, early_stopping]
   
    model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=100,
        )
    # model.fit_generator(train_gen,
    #         validation_data=valid_gen,
    #         steps_per_epoch=train_steps,
    #         validation_steps=valid_steps,
    #         epochs=epochs,
    #         callbacks=callbacks)
    
# !python3 run_modified.py

train steps: 3100 valid_steps 12
45359104/45351256 [==============================] - 5s 0us/step
Epoch 1/100
  15/3100 [..............................] - ETA: 7:13:40 - loss: 1.4288 - iou_score: 0.1819

KeyboardInterrupt: ignored